In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
# read raw data
raw = pd.read_parquet("shoppingcarts.parquet")

In [3]:
# compute sales per county and product for later use
sales_per_county = raw.groupby("county")["product_id"].count().rename("Bestellungen pro County")
sales_per_product = raw.groupby("product_id")["order_id"].count().rename("Bestellungen pro Produkt")

## Frage 1
###  Prüfen Sie, ob es belastbare Unterschiede im Wiederbestellverhalten zwischen den Produkten mit den product_ids 6217, 14778 und 23579 gibt. 

In [4]:
# filter to 6217, 14778 & 23579
# count reorders, compute fraction of reorders and avg time until reorder
# rename columns
df_q1 = raw[raw.product_id.isin([6217, 14778, 23579])].groupby("product_id")\
        .agg({"product_name": pd.Series.mode, "order_id": "count", "reordered": "mean", "days_since_prior_order": "mean"})\
        .rename(columns={"product_name": "Bezeichnung", "order_id": "Anzahl Bestellungen", "reordered": "Anteil Wiederbestellungen", "days_since_prior_order": "durchschn. Zeit bis Wiederbestellung"})

In [5]:
df_q1

,Bezeichnung,Anzahl Bestellungen,Anteil Wiederbestellungen,durchschn. Zeit bis Wiederbestellung
product_id,,,,
6217,Pumpkin Spice Protein Bar,46,0.456522,10.133333
14778,Organic Chocolate Chip Chewy Granola Bars,1329,0.619263,11.425059
23579,Peanut Butter Chocolate Chip Fruit & Nut Food Bar,1357,0.617539,11.003965


## Frage 2
### Schwankt die Popularität von Produkt 9390 zwischen den Regionen?

In [6]:
# filter to 9390, group by county and count orders
# sort and rename
df_q2 = raw[raw.product_id == 9390].groupby("county").agg({"order_id": "count"})\
        .sort_values(by="order_id")\
        .rename(columns={"order_id": "Anzahl Bestellungen"})
# compute fraction of orders
df_q2["Anteil Bestellungen"] = df_q2["Anzahl Bestellungen"] / int(df_q2.sum())
# compute share of product per county
df_q2["Anteil Bestellungen in County"] = df_q2["Anzahl Bestellungen"] / sales_per_county

In [7]:
df_q2

,Anzahl Bestellungen,Anteil Bestellungen,Anteil Bestellungen in County
county,,,
Alameda,1,0.001076,0.000063
Tulare,1,0.001076,0.000056
Sierra,1,0.001076,0.000038
San Diego,1,0.001076,0.000038
Del Norte,1,0.001076,0.000057
Napa,1,0.001076,0.000059
Tuolumne,2,0.002153,0.000090
Inyo,3,0.003229,0.000054
Siskiyou,3,0.003229,0.000038


## Frage 3
### Sind die Produkte 9390, 2713, 21883 und 16753 in den gleichen Regionen populär und unpopulär, oder unterscheiden sich die Muster?

In [8]:
# filter to 9390, 2713, 21883 & 16753, group by county and product id and count orders
# rename column
df_q3 = raw[raw.product_id.isin([9390, 2713, 21883, 16753])].groupby(["county", "product_id"]).agg({"order_id": "count"})\
        .rename(columns={"order_id": "Anzahl Bestellungen"})
# join sales per product
df_q3 = df_q3.merge(sales_per_product, left_index=True, right_index=True)
# compute fraction
df_q3["Anteil Bestellungen"] = df_q3["Anzahl Bestellungen"] / df_q3["Bestellungen pro Produkt"]

In [9]:
df_q3.sort_values(by="Anteil Bestellungen")

,,Anzahl Bestellungen,Bestellungen pro Produkt,Anteil Bestellungen
county,product_id,,,
Napa,9390,1,929,0.001076
Alameda,9390,1,929,0.001076
Del Norte,9390,1,929,0.001076
San Diego,9390,1,929,0.001076
Tulare,9390,1,929,0.001076
...,...,...,...,...
Mariposa,2713,22,184,0.119565
Ventura,21883,22,152,0.144737
Yuba,21883,23,152,0.151316


## Frage 4
### Welche Counties sind sich ähnlich in Hinblick auf den Produktmix?

In [10]:
# group by county and department and count orders
# pivot to reshape from long to wide
df_q4 = raw.groupby(["county", "department"], as_index=False)[["order_id"]].count()\
        .pivot(index="county", columns="department", values="order_id")

In [11]:
df_q4

department,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
county,,,,,,,,,,,,,,,,,,,,,
Alameda,103.0,124.0,538.0,1913.0,303.0,4.0,275.0,2526.0,392.0,199.0,...,321.0,51.0,243.0,10.0,8.0,749.0,210.0,42.0,5205.0,1932.0
Alpine,331.0,2239.0,4645.0,9434.0,3384.0,187.0,4132.0,22692.0,4154.0,3726.0,...,3503.0,1251.0,2843.0,366.0,407.0,7657.0,2462.0,452.0,32568.0,11073.0
Amador,681.0,710.0,3189.0,7816.0,2439.0,146.0,3404.0,15804.0,3145.0,2093.0,...,3027.0,704.0,2075.0,161.0,86.0,6045.0,1581.0,198.0,36091.0,9968.0
Butte,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
Calaveras,409.0,7192.0,19989.0,41760.0,10765.0,453.0,19197.0,88473.0,19688.0,9177.0,...,10818.0,3906.0,12935.0,1237.0,294.0,30094.0,7445.0,1767.0,150696.0,46604.0
Colusa,334.0,367.0,923.0,1920.0,863.0,248.0,997.0,5897.0,901.0,841.0,...,554.0,292.0,456.0,61.0,21.0,1573.0,351.0,29.0,11435.0,3307.0
Contra Costa,257.0,1276.0,3549.0,7350.0,1748.0,35.0,3403.0,13252.0,3175.0,3071.0,...,2326.0,937.0,2084.0,180.0,115.0,5674.0,979.0,585.0,24732.0,7765.0
Del Norte,428.0,93.0,893.0,1613.0,545.0,31.0,642.0,3363.0,509.0,268.0,...,817.0,215.0,322.0,21.0,39.0,1103.0,308.0,180.0,3590.0,1484.0
El Dorado,NaN,NaN,166.0,786.0,188.0,80.0,105.0,937.0,263.0,53.0,...,160.0,40.0,77.0,22.0,2.0,183.0,84.0,9.0,2475.0,1149.0


## Frage 5
### Welche der TOP 50-Produkte sind sich ähnlich in Hinblick auf die regionale Verteilung?

In [12]:
# compute top 50 products
top50 = sales_per_product.sort_values().tail(50).index
# filter to top 50 products, group by product and county and count orders
# pivot to reshape from long to wide
df_q5 = raw[raw["product_id"].isin(top50)].groupby(["product_id", "county"], as_index=False)[["order_id"]].count()\
        .pivot(index="county", columns="product_id", values="order_id")

In [13]:
df_q5

product_id,4605,4920,5077,5876,8277,8518,9076,10749,13176,16797,...,44359,44632,45007,45066,46979,47209,47626,47766,49235,49683
county,,,,,,,,,,,,,,,,,,,,,
Alameda,18.0,50.0,25.0,25.0,28.0,18.0,23.0,16.0,255.0,94.0,...,64.0,37.0,8.0,28.0,23.0,50.0,77.0,46.0,96.0,27.0
Alpine,225.0,279.0,226.0,242.0,400.0,215.0,222.0,358.0,1146.0,509.0,...,234.0,334.0,378.0,226.0,184.0,785.0,529.0,520.0,313.0,310.0
Amador,252.0,357.0,178.0,290.0,260.0,200.0,269.0,155.0,1630.0,892.0,...,188.0,186.0,302.0,254.0,161.0,610.0,647.0,517.0,466.0,331.0
Calaveras,1292.0,1437.0,1159.0,1491.0,1485.0,1221.0,802.0,1025.0,5637.0,2288.0,...,1120.0,1359.0,1729.0,1320.0,1136.0,3583.0,2376.0,3065.0,1216.0,1609.0
Colusa,86.0,117.0,46.0,127.0,156.0,96.0,40.0,66.0,538.0,164.0,...,78.0,56.0,106.0,66.0,61.0,220.0,118.0,210.0,108.0,98.0
Contra Costa,177.0,212.0,185.0,318.0,212.0,185.0,107.0,136.0,1027.0,255.0,...,216.0,203.0,231.0,222.0,143.0,868.0,332.0,375.0,148.0,202.0
Del Norte,40.0,52.0,40.0,5.0,60.0,7.0,42.0,6.0,198.0,58.0,...,23.0,9.0,9.0,4.0,26.0,30.0,87.0,54.0,30.0,21.0
El Dorado,15.0,29.0,NaN,4.0,4.0,13.0,16.0,3.0,148.0,46.0,...,2.0,45.0,22.0,6.0,21.0,40.0,15.0,51.0,62.0,23.0
Fresno,891.0,948.0,623.0,1099.0,1028.0,912.0,767.0,843.0,4804.0,1579.0,...,959.0,646.0,1539.0,1019.0,768.0,2684.0,1879.0,2693.0,1081.0,1291.0
